# Homework
## Dataset
In this homework, we will continue the New York City Airbnb Open Data.

We'll keep working with the `price` variable, and we'll transform it to a classification task.

## Features
For the rest of the homework, you'll need to use the features from the previous homework with additional two `neighbourhood_group` and `room_type`. So the whole feature set will be set as follows:

* `neighbourhood_group`,
* `room_type`,
* `latitude`,
* `longitude`,
* `price`,
* `minimum_nights`,
* `number_of_reviews`,
* `reviews_per_month`,
* `calculated_host_listings_count`,
* `availability_365`

Select only them and fill in the missing values with 0.

In [1]:
import numpy as np
import pandas as pd

columns = [
    'neighbourhood_group',
    'room_type',
    'latitude',
    'longitude',
    'price',
    'minimum_nights',
    'number_of_reviews',
    'reviews_per_month',
    'calculated_host_listings_count',
    'availability_365'    
]
df = pd.read_csv('AB_NYC_2019.csv')[columns].fillna(0)
df.head()

,neighbourhood_group,room_type,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Brooklyn,Private room,40.64749,-73.97237,149,1,9,0.21,6,365
1,Manhattan,Entire home/apt,40.75362,-73.98377,225,1,45,0.38,2,355
2,Manhattan,Private room,40.80902,-73.94190,150,3,0,0.00,1,365
3,Brooklyn,Entire home/apt,40.68514,-73.95976,89,1,270,4.64,1,194
4,Manhattan,Entire home/apt,40.79851,-73.94399,80,10,9,0.10,1,0


## Question 1
What is the most frequent observation (mode) for the column `neighbourhood_group`?

In [2]:
df.groupby('neighbourhood_group').size()

neighbourhood_group
Bronx             1091
Brooklyn         20104
Manhattan        21661
Queens            5666
Staten Island      373
dtype: int64

### Split the data

* Split your data in train/val/test sets, with 60%/20%/20% distribution.
* Use Scikit-Learn for that (the `train_test_split` function) and set the seed to 42.
* Make sure that the target value (`'price'`) is not in your dataframe.

In [3]:
from sklearn.model_selection import train_test_split

def split_data(df, seed=42):
    train, test = train_test_split(df, test_size=0.2, random_state=seed)
    train, val = train_test_split(test, test_size=0.25, random_state=seed)
    
    dfs = [[d.drop('price', axis=1).reset_index(drop=True), d['price'].reset_index(drop=True)]
          for d in [train, val, test]]
    
    return [item for pair in dfs for item in pair]

In [4]:
trainX, trainy, valX, valy, testX, testy = split_data(df)
trainX.head()

,neighbourhood_group,room_type,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Brooklyn,Private room,40.62834,-73.90506,2,37,2.19,1,67
1,Manhattan,Entire home/apt,40.73476,-74.00100,1,7,0.12,1,0
2,Manhattan,Private room,40.85612,-73.93024,1,50,1.32,1,90
3,Manhattan,Private room,40.73083,-73.99430,4,0,0.00,1,0
4,Queens,Shared room,40.71540,-73.91148,1,5,2.68,1,89


## Question 2

* Create the correlation matrix for the numerical features of your train dataset.
  * In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.
* What are the two features that have the biggest correlation in this dataset?


In [5]:
trainX.corr()

,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
latitude,1.000000,0.109087,0.022696,-0.026649,-0.037324,0.020536,-0.022594
longitude,0.109087,1.000000,-0.056032,0.053435,0.142865,-0.103358,0.084891
minimum_nights,0.022696,-0.056032,1.000000,-0.097775,-0.150102,0.177761,0.174556
number_of_reviews,-0.026649,0.053435,-0.097775,1.000000,0.605155,-0.070350,0.168153
reviews_per_month,-0.037324,0.142865,-0.150102,0.605155,1.000000,-0.051401,0.163345
calculated_host_listings_count,0.020536,-0.103358,0.177761,-0.070350,-0.051401,1.000000,0.228664
availability_365,-0.022594,0.084891,0.174556,0.168153,0.163345,0.228664,1.000000


In [6]:
trainX.corr().replace({1: 0}).max().sort_values(ascending=False)

number_of_reviews                 0.605155
reviews_per_month                 0.605155
calculated_host_listings_count    0.228664
availability_365                  0.228664
minimum_nights                    0.177761
longitude                         0.142865
latitude                          0.109087
dtype: float64

In [7]:
trainX.corr().replace({1: 0}).min().sort_values(ascending=True)

minimum_nights                   -0.150102
reviews_per_month                -0.150102
longitude                        -0.103358
calculated_host_listings_count   -0.103358
number_of_reviews                -0.097775
latitude                         -0.037324
availability_365                 -0.022594
dtype: float64

Two features with largest correlation are `number_of_reviews` and `reviews_per_month`.

### Make price binary

* We need to turn the price variable from numeric into binary.
* Let's create a variable `above_average` which is one if the price is above (or equal to) `152`.

In [8]:
train_above_average, val_above_average, test_above_average = [(y >= 152).astype('int') for y in [trainy, valy, testy]]

## Question 3

* Calculate the mutual information score for the two categorical variables that we have. Use the training set only.
* Which of these two variables has bigger score?
* Round it to 2 decimal digits using `round(score, 2)`

In [9]:
from sklearn.metrics import mutual_info_score

round(mutual_info_score(trainX.neighbourhood_group, train_above_average), 2)

0.04

In [10]:
round(mutual_info_score(trainX.room_type, train_above_average), 2)

0.14

`room_type` has a bigger score.

## Question 4

* Now let's train a logistic regression
* Remember that we have two categorical variables in the data. Include them using one-hot encoding.
* Fit the model on the training dataset.
  * To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
  * model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
* Calculate the accuracy on the validation dataset and rount it to 2 decimal digits.

In [11]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

CAT_VARS = ['neighbourhood_group', 'room_type']
NUM_VARS = list(filter(lambda s: s not in CAT_VARS, trainX.columns))

def train_dictvectorizer(df, cat_vars):
    dv = DictVectorizer(sparse=False)
    dv.fit(trainX[cat_vars].to_dict(orient='records'))
    return dv

def transform(df, dv, col_drop):
    nums = df[NUM_VARS].drop(col_drop, axis=1, errors='ignore')
    cats = dv.transform(df[CAT_VARS].drop(col_drop, axis=1, errors='ignore').to_dict(orient='records'))
    return np.c_[nums, cats]


def train_model(dfX, dfy, dv, col_drop=''):
    dfX_transformed = transform(dfX, dv, col_drop)
    model = LogisticRegression(solver='liblinear', C=1.0, random_state=42)
    model.fit(dfX_transformed, dfy)
    
    return model

def score_on_val(model, valX, valy, dv, col_drop=''):
    valX_transformed = transform(valX, dv, col_drop)
    predictions = model.predict(valX_transformed)
    return round((predictions == valy).mean(), 2)


dv = train_dictvectorizer(trainX, CAT_VARS)
model = train_model(trainX, train_above_average, dv)
original_score = score_on_val(model, valX, val_above_average, dv)
original_score

0.8

## Question 5

* We have 9 features: 7 numerical features and 2 categorical.
* Let's find the least useful one using the feature elimination technique.
* Train a model with all these features (using the same parameters as in Q4).
* Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
* For each feature, calculate the difference between the original accuracy and the accuracy without the feature.
* Which of following feature has the smallest difference?
  * `neighbourhood_group`
  * `room_type`
  * `number_of_reviews`
  * `reviews_per_month`

> note: the difference doesn't have to be positive

In [12]:
scores = {}


for col in df.columns:
    dv = train_dictvectorizer(trainX.drop(col, axis=1, errors='ignore'), list(filter(lambda s: s != col, CAT_VARS)))
    model = train_model(trainX, train_above_average, dv, col)
    scores[col] = score_on_val(model, valX, val_above_average, dv, col)

{col: round(abs(original_score - score), 2) for (col, score) in scores.items()}

{'neighbourhood_group': 0.06,
 'room_type': 0.07,
 'latitude': 0.0,
 'longitude': 0.0,
 'price': 0.0,
 'minimum_nights': 0.01,
 'number_of_reviews': 0.01,
 'reviews_per_month': 0.01,
 'calculated_host_listings_count': 0.0,
 'availability_365': 0.01}

## Question 6

* For this question, we'll see how to use a linear regression model from Scikit-Learn
* We'll need to use the original column `'price'`. Apply the logarithmic transformation to this column.
* Fit the Ridge regression model on the training data.
* This model has a parameter `alpha`. Let's try the following values: `[0, 0.01, 0.1, 1, 10]`
* Which of these alphas leads to the best RMSE on the validation set? Round your RMSE scores to 3 decimal digits.

If there are multiple options, select the smallest `alpha`.

In [13]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

trainy = np.log1p(trainy)
valy = np.log1p(valy)
testy = np.log1p(testy)
dv = train_dictvectorizer(trainX, CAT_VARS)
scores = []

for alpha in [0, 0.01, 0.1, 1, 10]:
    model = Ridge(alpha=alpha, random_state=42)
    model.fit(transform(trainX, dv, ''), trainy)
    
    predictions = model.predict(transform(valX, dv, ''))
    rmse = np.sqrt(mean_squared_error(predictions, valy))
    scores.append({'alpha': alpha, 'rmse': round(rmse, 3)})
    
scores

[{'alpha': 0, 'rmse': 0.478},
 {'alpha': 0.01, 'rmse': 0.478},
 {'alpha': 0.1, 'rmse': 0.478},
 {'alpha': 1, 'rmse': 0.478},
 {'alpha': 10, 'rmse': 0.482}]